# Life Cycle Cost analysis demonstrator
## *version 1.5.2*   -  executing annual life cycle cost analysis across unified asset register
### Conor Shaw
### conor.shaw@ucdconnect.ie
### Date: 23.08.2024

# Preamble & import packages

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import datetime
import time

# specify file path
path = 'data/'

# today's date 
today = 2019

# user inputs

### represents the life cycle analysis scope


In [2]:
### import user input data, create, assign values and print variables for each column

# user input dataframe
user_input = pd.read_excel(path+'user_input.xlsx')
for col_name in user_input.columns:
    globals()[col_name] = user_input[col_name][0]
    
# print message + variables
print('THE VARIABLES AND ASSOCIATED VALUES EXTRACTED FROM THE USER INPUT DATABASE ARE:')
print(' ')
# Loop through column headers
for col_name in user_input.columns:
    # Get the value of the column
    col_value = user_input[col_name][0]
    # Print the variable name and its value
    print(f"{col_name}: {col_value}")
    # Assign the value to the variable with the same name
    globals()[col_name] = col_value

THE VARIABLES AND ASSOCIATED VALUES EXTRACTED FROM THE USER INPUT DATABASE ARE:
 
study_period_start: 2019
study_period_end: 2092
study_period: 73
discount_rate: 0.0416
inflation_rate: 0.02


# asset register
### input asset data extracted from unified graph database

In [3]:
# create asset register dataframe from excel file
asset_register = pd.read_excel(path+'asset_register.xlsx')
asset_register

asset_id  installation_date  lifetime  initial_cost  replacement_cost  \
0        31126               2003        40      100000.0          100000.0   
1        31127               2003        40      100000.0          100000.0   
2        31128               2003        40      100000.0          100000.0   
3        31129               2003        40      100000.0          100000.0   
4        31146               2003        40      100000.0          100000.0   
...        ...                ...       ...           ...               ...   
3924  82359604               2017         7        1200.0            1200.0   
3925  82359605               2017         7        1200.0            1200.0   
3926  82359634               2017         7        1200.0            1200.0   
3927  82359635               2017         7        1200.0            1200.0   
3928  82349091               2012        25       25000.0           25000.0   

      annual_energy  annual_maintenance  renovation_cost  renovation_frequency  
0            26.280          167.396483           5000.0                     4  
1            26.280          167.396483           5000.0                     4  
2            26.280          167.396483           5000.0                     4  
3            26.280          167.396483           5000.0                     4  
4            26.280          167.396483           5000.0                     4  
...             ...                 ...              ...                   ...  
3924         65.700           24.451803              0.0                     0  
3925         14.454           24.451803              0.0                     0  
3926         13.140           24.451803              0.0                     0  
3927        109.500           24.451803              0.0                     0  
3928       2952.850           16.301202           2500.0                     5  

[3929 rows x 9 columns]

# NPV analysis
### Function to calculate Net Present Value (NPV)

In [ ]:
def calculate_asset_NPV(asset_id, installation_date, lifetime, initial_cost, replacement_cost, annual_energy, annual_maintenance, renovation_cost, renovation_frequency):
    
### DATAFRAME FROM STUDY PERIOD AND ADD ANNUAL ENERGY AND MAINTENANCE COSTS
    
    # Create a list of years from study_period_start to study_period_start + study_period + 1
    years = list(range(study_period_start, study_period_start + study_period + 1))
    # Create a DataFrame with the years
    years_df = pd.DataFrame({'year': years})
    
    # add 'energy cost' column and fill with the value of annual_energy rounded to 2 decimal places
    years_df['energy cost'] = round(annual_energy, 2)
    
    # add 'maint cost' column and fill with the value of annual_maintenance rounded to 2 decimal places
    years_df['maint cost'] = round(annual_maintenance, 2)
    
    #create copy of 'years_df' called 'study_period_df' to work with previous code
    study_period_df = years_df.copy()
    
### AQUISITION, RENOVATION AND REPLACEMENT COSTS
    
    # add acquisition cost column to right of 'year' column
    study_period_df.insert(study_period_df.columns.get_loc('year')+1, 'acquisition cost', 0.0)
    # update acquisition cost at installation date
    study_period_df.loc[study_period_df['year'] == installation_date, 'acquisition cost'] = initial_cost
    
    # Add a new column 'renovation cost' and set all values to 0.0
    study_period_df['renovation cost'] = 0.0
    # uses a while loop to add 'renovation_frequency' to the 'installation_date' and add 'renovation_cost' to corresponding years
    # stops when the value exceeds the 'study_period_end'
    
    # ensure that renovation_frequency is more that 0
    if renovation_frequency > 0:
        # Loop through the renovation years
        renovation_year = installation_date + renovation_frequency
        while renovation_year <= study_period_end:
            # Check if the renovation year is in the study_period_df dataframe
            if renovation_year in study_period_df['year'].values:
                # Update the renovation cost for that year
                study_period_df.loc[study_period_df['year'] == renovation_year, 'renovation cost'] += renovation_cost
            # increment the renovation year
            renovation_year += renovation_frequency
    
    # Add a new column 'replacement cost' and set all values to 0.0
    study_period_df['replacement cost'] = 0.0
    # uses a while loop to add 'lifetime' to the 'installation_date' and add 'replacement_cost' to corresponding years
    # stops when the value exceeds the 'study_period_end'
    # includes a step to delete overlapping values in the 'renovation cost' column of the replacement years.
    # Loop through the replacement years
    replacement_year = installation_date + lifetime 
    while replacement_year <= study_period_end:
        # Check if the replacement year is in the study_period_df dataframe
        if replacement_year in study_period_df['year'].values:
            # Update the replacement cost for that year
            study_period_df.loc[study_period_df['year'] == replacement_year, 'replacement cost'] += replacement_cost
            # Check and replace non-zero values in the 'renovation cost' column
            for i in study_period_df[study_period_df['year'] == replacement_year].index:
                if study_period_df.loc[i, 'renovation cost'] != 0.0:
                    study_period_df.loc[i, 'renovation cost'] = 0.0
        # increment the renovation year
        replacement_year += lifetime
    
### TOTAL COSTS, INFLATION AND NPV
    
    # create a 'total cost' column, sum the values to left excluding the year column
    study_period_df['total cost'] = study_period_df.iloc[:, 1:].sum(axis=1)
    
    # add a new column 'total + infl.' and set all values to 0.0
    study_period_df['total + infl.'] = 0.0
    # loop through dataframe and apply inflation to years after 'today'
    for i, row in study_period_df.iterrows():
        # check if the year is after 'today'
        if row['year'] >= today:
            # calculate the inflation factor
            infl_factor = (1 + inflation_rate) ** (row['year'] - today)
            # apply the inflation to the 'total' column
            study_period_df.at[i, 'total + infl.'] = round(row['total cost'] * infl_factor, 2)
        else:
            # for years prior to 'today', use the 'total cost' value unchanged
            study_period_df.at[i, 'total + infl.'] = round(row['total cost'], 2)
    
    # add a new column 'NPV' and set all values to 0.0
    study_period_df['NPV'] = 0.0
    # get the index of the 'today' year in the dataframe
    today_index = study_period_df.index[study_period_df['year'] == today][0]
    # loop through the dataframe
    for i, row in study_period_df.iterrows():
        # calculate the present value of the total cost
        cash_flow = row['total + infl.']
        if i <= today_index:  # years prior to or including 'today' year
            study_period_df.at[i, 'NPV'] = cash_flow
        else:
            PV = cash_flow / ((1 + discount_rate) ** (i-today_index))
            study_period_df.at[i, 'NPV'] = round(PV, 2)

    # drop all but year and npv in new dataframe
    time_npv = study_period_df.loc[:, ['year', 'NPV']]
    # transpose so years are column headers
    time_npv = time_npv.set_index('year').T
    # set index as 'asset_id' variable and fill NPV values for year columns
    time_npv = time_npv.rename_axis(index={ 'NPV': asset_id }).rename(index={ 'NPV': asset_id })
    # set index head as 'asset_id'
    time_npv.index.name = 'asset_id'
    
    # return the new dataframe 
    return time_npv

In [ ]:
### TO DEMONSTRATE 'calculate_asset_NPV' FUNCTION ON INDIVIDUAL ASSET SELECTED

index_select = 8

# Create variables with the same name as the column headers and assign them the value from the first row
for col_name in asset_register.columns:
    globals()[col_name] = asset_register[col_name][index_select]
# print message
print('THE VARIABLES AND ASSOCIATED VALUES EXTRACTED FROM THE ASSET REGISTER DATABASE AT INDEX [' + str(index_select) + '] ARE:')
print(' ')

# Loop through column headers
for col_name in asset_register.columns:
    # Get the value of the column
    col_value = asset_register[col_name][index_select]
    # Print the variable name and its value
    print(f"{col_name}: {col_value}")
    # Assign the value to the variable with the same name
    globals()[col_name] = col_value 

print(' ')
print('THE calculate_asset_NPv FUNCTION APPLIED TO THIS INDEX ROW RETURNS:')
print(' ')
    
calculate_asset_NPV(asset_id, installation_date, lifetime, initial_cost, replacement_cost, annual_energy, annual_maintenance, renovation_cost, renovation_frequency)

THE VARIABLES AND ASSOCIATED VALUES EXTRACTED FROM THE ASSET REGISTER DATABASE AT INDEX [8] ARE:
 
asset_id: 31154
installation_date: 1999
lifetime: 40
initial_cost: 100000.0
replacement_cost: 100000.0
annual_energy: 26.28
annual_maintenance: 167.39648285591252
renovation_cost: 5000.0
renovation_frequency: 4
 
THE calculate_asset_NPv FUNCTION APPLIED TO THIS INDEX ROW RETURNS:
 


year         2019    2020    2021    2022     2023    2024   2025    2026  \
asset_id                                                                    
31154     5193.68  189.66  185.73  181.87  4776.09  174.42  170.8  167.26   

year         2027    2028  ...     2083   2084   2085   2086     2087   2088  \
asset_id                   ...                                                 
31154     4392.06  160.39  ...  1358.39  49.61  48.58  47.57  1249.17  45.62   

year       2089   2090     2091   2092  
asset_id                                
31154     44.67  43.75  1148.73  41.95  

[1 rows x 74 columns]

### Function to run asset register and aggregate into new dataframe (NPV)

In [ ]:
def calculate_asset_NPV_by_group(asset_register, inflation_rate, discount_rate, today, study_period_start, study_period_end, study_period):
    # empty dataframe
    result_df = pd.DataFrame()
    # group inputs by asset_id column
    grouped_df = asset_register.groupby('asset_id')
    # loop through groups
    for name, group in grouped_df:
        # assign variables
        asset_id = str(group.iloc[0]['asset_id']).split('.')[0] # convert to string and remove .0
        installation_date = group.iloc[0]['installation_date']
        lifetime = group.iloc[0]['lifetime']
        initial_cost = group.iloc[0]['initial_cost']
        replacement_cost = group.iloc[0]['replacement_cost']
        annual_energy = group.iloc[0]['annual_energy']
        annual_maintenance = group.iloc[0]['annual_maintenance']
        renovation_cost = group.iloc[0]['renovation_cost']
        renovation_frequency = group.iloc[0]['renovation_frequency']
        # call the 'calculate_asset_NPV' function
        sum_df = calculate_asset_NPV(asset_id, installation_date, lifetime, initial_cost, replacement_cost, annual_energy, annual_maintenance, renovation_cost, renovation_frequency)
        #set index of sum_df to a single value 'asset_id' and names the index as 'asset_id'.
        sum_df.index = pd.Index([asset_id], name='asset_id')
        # concatenate the results to new dataframe 
        result_df = pd.concat([result_df, sum_df])
    # resets index numerically (but has as seperate column)
    result_df.reset_index(drop=False, inplace=True)
    # sets the index to 'asset_id'
    result_df.set_index('asset_id', inplace=True)
    return result_df

### Call the NPV analysis functions

In [ ]:
# get the current time before running the script
start_time = time.time()

# call the calculate_asset_NPV_by_group function and assign to new dataframe
aggregated_npv = calculate_asset_NPV_by_group(asset_register, inflation_rate, discount_rate, today, study_period_start, study_period_end, study_period)

# calculate the total time it took to run the script
total_time = time.time() - start_time

# print the total time
print("Total time taken to run the script: %s seconds" % total_time)

Total time taken to run the script: 116.61018013954163 seconds


In [ ]:
# return results
aggregated_npv

year           2019      2020      2021     2022     2023      2024     2025  \
asset_id                                                                       
3576        8378.49   8204.74   8034.60  7867.98  7704.82  12047.67  7388.58   
3577        8378.49   8204.74  12829.37  7867.98  7704.82   7545.04  7388.58   
3585        8378.49  13101.06   8034.60  7867.98  7704.82   7545.04  7388.58   
3587        8378.49   8204.74   8034.60  7867.98  7704.82  12047.67  7388.58   
3605       13378.49   8204.74   8034.60  7867.98  7704.82   7545.04  7388.58   
...             ...       ...       ...      ...      ...       ...      ...   
82359470A     64.67     63.33     62.01   107.68    59.47     58.24    57.03   
82359479A     64.67     63.33     62.01   107.68    59.47     58.24    57.03   
82359490A     64.67     63.33     62.01   107.68    59.47     58.24    57.03   
82362189A    393.95    385.78    377.78   369.94   362.27    354.76   347.40   
82362214A    224.10    219.45    214.90   210.45  1585.47    201.80   197.62   

year           2026      2027       2028  ...     2083     2084     2085  \
asset_id                                  ...                              
3576        7235.36   7085.32  213968.16  ...  2191.36  2145.92  2101.42   
3577        7235.36   7085.32   11078.98  ...  2191.36  3426.53  2101.42   
3585        7235.36  11313.59    6938.39  ...  3499.10  2145.92  2101.42   
3587        7235.36   7085.32  213968.16  ...  2191.36  2145.92  2101.42   
3605       11553.18   7085.32    6938.39  ...  2191.36  2145.92  2101.42   
...             ...       ...        ...  ...      ...      ...      ...   
82359470A     55.85     96.97      53.56  ...    16.91    16.56    16.22   
82359479A     55.85     96.97      53.56  ...    16.91    16.56    16.22   
82359490A     55.85     96.97      53.56  ...    16.91    16.56    16.22   
82362189A    340.20    333.15     326.24  ...   103.04   100.90    98.81   
82362214A    193.52    189.51     682.46  ...   450.93    57.40    56.21   

year           2086     2087     2088     2089     2090      2091      2092  
asset_id                                                                     
3576        2057.84  3217.75  1973.38  1932.46  1892.38   1853.14   1814.71  
3577       63460.40  2015.17  1973.38  1932.46  1892.38   2959.03   1814.71  
3585        2057.84  2015.17  1973.38  1932.46  3021.69   1853.14  55962.61  
3587        2057.84  3217.75  1973.38  1932.46  1892.38   1853.14   1814.71  
3605        2057.84  2015.17  1973.38  3085.68  1892.38   1853.14   1814.71  
...             ...      ...      ...      ...      ...       ...       ...  
82359470A     15.88    27.58    15.23    14.92    14.61     14.30  54161.91  
82359479A     15.88    27.58    15.23    14.92    14.61     14.30  54161.91  
82359490A     15.88    27.58    15.23    14.92    14.61     14.30  54161.91  
82362189A     96.76    94.75    92.79    90.86    88.98  11146.05     85.33  
82362214A     55.04    53.90   194.10    51.69    50.62     49.57     48.54  

[3929 rows x 74 columns]

### extracting cashflow table (to be uploaded to 'enrich' graph database)


In [ ]:
# Sum all the years for each asset_id and add as a new column 'total_npv'
aggregated_npv['total_npv'] = aggregated_npv.sum(axis=1)

# Reorder columns to place 'total_npv' right after 'asset_id'
columns = ['total_npv'] + [col for col in aggregated_npv.columns if col != 'total_npv']
aggregated_npv = aggregated_npv[columns]

# Display the updated dataframe
aggregated_npv

year       total_npv      2019      2020      2021     2022     2023  \
asset_id                                                               
3576       745970.22   8378.49   8204.74   8034.60  7867.98  7704.82   
3577       686639.26   8378.49   8204.74  12829.37  7867.98  7704.82   
3585       647045.54   8378.49  13101.06   8034.60  7867.98  7704.82   
3587       745970.22   8378.49   8204.74   8034.60  7867.98  7704.82   
3605       733418.10  13378.49   8204.74   8034.60  7867.98  7704.82   
...              ...       ...       ...       ...      ...      ...   
82359470A  561963.30     64.67     63.33     62.01   107.68    59.47   
82359479A  561963.30     64.67     63.33     62.01   107.68    59.47   
82359490A  561963.30     64.67     63.33     62.01   107.68    59.47   
82362189A   78295.79    393.95    385.78    377.78   369.94   362.27   
82362214A   16150.00    224.10    219.45    214.90   210.45  1585.47   

year           2024     2025      2026      2027  ...     2083     2084  \
asset_id                                          ...                     
3576       12047.67  7388.58   7235.36   7085.32  ...  2191.36  2145.92   
3577        7545.04  7388.58   7235.36   7085.32  ...  2191.36  3426.53   
3585        7545.04  7388.58   7235.36  11313.59  ...  3499.10  2145.92   
3587       12047.67  7388.58   7235.36   7085.32  ...  2191.36  2145.92   
3605        7545.04  7388.58  11553.18   7085.32  ...  2191.36  2145.92   
...             ...      ...       ...       ...  ...      ...      ...   
82359470A     58.24    57.03     55.85     96.97  ...    16.91    16.56   
82359479A     58.24    57.03     55.85     96.97  ...    16.91    16.56   
82359490A     58.24    57.03     55.85     96.97  ...    16.91    16.56   
82362189A    354.76   347.40    340.20    333.15  ...   103.04   100.90   
82362214A    201.80   197.62    193.52    189.51  ...   450.93    57.40   

year          2085      2086     2087     2088     2089     2090      2091  \
asset_id                                                                     
3576       2101.42   2057.84  3217.75  1973.38  1932.46  1892.38   1853.14   
3577       2101.42  63460.40  2015.17  1973.38  1932.46  1892.38   2959.03   
3585       2101.42   2057.84  2015.17  1973.38  1932.46  3021.69   1853.14   
3587       2101.42   2057.84  3217.75  1973.38  1932.46  1892.38   1853.14   
3605       2101.42   2057.84  2015.17  1973.38  3085.68  1892.38   1853.14   
...            ...       ...      ...      ...      ...      ...       ...   
82359470A    16.22     15.88    27.58    15.23    14.92    14.61     14.30   
82359479A    16.22     15.88    27.58    15.23    14.92    14.61     14.30   
82359490A    16.22     15.88    27.58    15.23    14.92    14.61     14.30   
82362189A    98.81     96.76    94.75    92.79    90.86    88.98  11146.05   
82362214A    56.21     55.04    53.90   194.10    51.69    50.62     49.57   

year           2092  
asset_id             
3576        1814.71  
3577        1814.71  
3585       55962.61  
3587        1814.71  
3605        1814.71  
...             ...  
82359470A  54161.91  
82359479A  54161.91  
82359490A  54161.91  
82362189A     85.33  
82362214A     48.54  

[3929 rows x 75 columns]

In [ ]:
# Copy the original DataFrame to a new DataFrame
new_df = aggregated_npv.copy()

# Ensure 'year' is a column and not an index
if new_df.index.name == 'year':
    new_df.reset_index(inplace=True)

# Calculate the annual cashflow, excluding the 'total_npv' row
# Ensure 'total_npv' is not included in the summation
if 'total_npv' in new_df.index:
    df_no_total_npv = new_df.drop('total_npv', errors='ignore')
else:
    df_no_total_npv = new_df

# Perform summation across rows
annual_cashflow = df_no_total_npv.drop('total_npv', errors='ignore').sum(axis=0)

# Insert the 'annual cashflow' row
new_df.loc['annual cashflow'] = annual_cashflow

# Reorder columns to place 'total_npv' beside 'asset_id'
columns_order = ['total_npv'] + [col for col in new_df.columns if col != 'total_npv']
new_df = new_df[columns_order]

# Display the updated DataFrame

new_df



year                total_npv         2019         2020        2021  \
asset_id                                                              
3576             7.459702e+05      8378.49      8204.74     8034.60   
3577             6.866393e+05      8378.49      8204.74    12829.37   
3585             6.470455e+05      8378.49     13101.06     8034.60   
3587             7.459702e+05      8378.49      8204.74     8034.60   
3605             7.334181e+05     13378.49      8204.74     8034.60   
...                       ...          ...          ...         ...   
82359479A        5.619633e+05        64.67        63.33       62.01   
82359490A        5.619633e+05        64.67        63.33       62.01   
82362189A        7.829579e+04       393.95       385.78      377.78   
82362214A        1.615000e+04       224.10       219.45      214.90   
annual cashflow  1.044342e+09  20680896.83  18492090.85  9983366.48   

year                    2022         2023         2024         2025  \
asset_id                                                              
3576                 7867.98      7704.82     12047.67      7388.58   
3577                 7867.98      7704.82      7545.04      7388.58   
3585                 7867.98      7704.82      7545.04      7388.58   
3587                 7867.98      7704.82     12047.67      7388.58   
3605                 7867.98      7704.82      7545.04      7388.58   
...                      ...          ...          ...          ...   
82359479A             107.68        59.47        58.24        57.03   
82359490A             107.68        59.47        58.24        57.03   
82362189A             369.94       362.27       354.76       347.40   
82362214A             210.45      1585.47       201.80       197.62   
annual cashflow  17850138.28  30823053.61  11107099.68  16028462.42   

year                    2026         2027  ...        2083        2084  \
asset_id                                   ...                           
3576                 7235.36      7085.32  ...     2191.36     2145.92   
3577                 7235.36      7085.32  ...     2191.36     3426.53   
3585                 7235.36     11313.59  ...     3499.10     2145.92   
3587                 7235.36      7085.32  ...     2191.36     2145.92   
3605                11553.18      7085.32  ...     2191.36     2145.92   
...                      ...          ...  ...         ...         ...   
82359479A              55.85        96.97  ...       16.91       16.56   
82359490A              55.85        96.97  ...       16.91       16.56   
82362189A             340.20       333.15  ...      103.04      100.90   
82362214A             193.52       189.51  ...      450.93       57.40   
annual cashflow  43873214.32  38171165.36  ...  9365142.80  3095528.59   

year                   2085         2086         2087         2088  \
asset_id                                                             
3576                2101.42      2057.84      3217.75      1973.38   
3577                2101.42     63460.40      2015.17      1973.38   
3585                2101.42      2057.84      2015.17      1973.38   
3587                2101.42      2057.84      3217.75      1973.38   
3605                2101.42      2057.84      2015.17      1973.38   
...                     ...          ...          ...          ...   
82359479A             16.22        15.88        27.58        15.23   
82359490A             16.22        15.88        27.58        15.23   
82362189A             98.81        96.76        94.75        92.79   
82362214A             56.21        55.04        53.90       194.10   
annual cashflow  4157998.38  12920834.03  11190910.12  10249987.03   

year                   2089        2090        2091         2092  
asset_id                                                          
3576                1932.46     1892.38     1853.14      1814.71  
3577                1932.46     1892.38     2959.03      1814.71  
3585      

In [ ]:
# Assuming new_df is your DataFrame and you want to save it to an Excel file
file_name = 'LCCanalysis_output.xlsx'  # Specify your desired file name
sheet_name = 'Sheet1'  # Optional: specify the sheet name if you want

# Write the DataFrame to an Excel file
new_df.to_excel(file_name, sheet_name=sheet_name)

print(f'DataFrame has been written to {file_name}')

DataFrame has been written to LCCanalysis_output.xlsx
